# Telco customer churn data analysis 

This Jupyter notebook explores Telco churn data available on [Kaggle](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

In [3]:
#reading the csv file with data
df = pd.read_csv(r"C:\Users\Paulina\Desktop\Projects_Git\Telcom_churn\data\Telco_churn.csv")

## Data exploration

This part explores the columns and the values included in it.

In [5]:
df.info() #shows column entry types and the number of filled rows in each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


We can see that there are numerous columns with Yes/No values - I will be chnaging it into numerical values 0 and 1 during data cleaning. Any machine learning models will not be able to use categorical values so it's better to change them. Additionally, TotalCharges is registered as an object but they look like floating numbers. This will be changed as well to allow calculations. 

I can anticipate that there will be probably no missing values after seeing the Non-Null count in `.head()`, but let's check it to be sure.

In [9]:
#checking for missing values
print('Total number of missing values:')
print(df.isna().sum())

Total number of missing values:
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


Let's explore unique values for our cathegorical columns, because they may contain more than just `'Yes'` and `'No'` entries. 

In [11]:
cat_cols = df.select_dtypes(include = 'object').columns

for col in cat_cols:
    uniques = df[col].unique()
    print(f'Column:{col}')
    print(f'Unique values ({len(uniques)}):{uniques}')
    print('-'*40)

Column:customerID
Unique values (7043):['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
----------------------------------------
Column:gender
Unique values (2):['Female' 'Male']
----------------------------------------
Column:Partner
Unique values (2):['Yes' 'No']
----------------------------------------
Column:Dependents
Unique values (2):['No' 'Yes']
----------------------------------------
Column:PhoneService
Unique values (2):['No' 'Yes']
----------------------------------------
Column:MultipleLines
Unique values (3):['No phone service' 'No' 'Yes']
----------------------------------------
Column:InternetService
Unique values (3):['DSL' 'Fiber optic' 'No']
----------------------------------------
Column:OnlineSecurity
Unique values (3):['No' 'Yes' 'No internet service']
----------------------------------------
Column:OnlineBackup
Unique values (3):['Yes' 'No' 'No internet service']
----------------------------------------
Column:DeviceProtection


## Data cleaning

Includes changing datatypes and encoding categorical variables for modelling. Let's change all 'Yes' into 1 and 'No' into 0. 

In [13]:
# changing the string entires into numeric values with errors showing as NaN.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [14]:
# creating a copy of dataframe to have a backup before implementing big changes
# especially useful for data visualizations in Tableau because we retain string values instead of numerical ones
df_copy = df.copy()

In [15]:
change_yes_no = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
change_yes_no_nis = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']


# changing the yes into 1 and No into 0 in columns that only have those 2 unique entries:
for col in change_yes_no:
    df[col] = df[col].replace('Yes', 1)
    df[col] = df[col].replace('No', 0)

# changing Yes into 1, No into 0 and No internet service into 2 
for col in change_yes_no_nis:
    df[col] = df[col].replace('Yes', 1)
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('No internet service', 2)

# changing female into 0 and male into 1
df['gender'] = df['gender'].replace('Female', 0)
df['gender'] = df['gender'].replace('Male', 1)

# changing the MultipleLines in similiar way
df['MultipleLines'] = df['MultipleLines'].replace('No', 0)
df['MultipleLines'] = df['MultipleLines'].replace('Yes', 1)
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 2)


In [16]:
cat_cols = df.select_dtypes(include = 'object').columns

for col in cat_cols:
    uniques = df[col].unique()
    print(f'Column:{col}')
    print(f'Unique values ({len(uniques)}):{uniques}')
    print('-'*40)

Column:customerID
Unique values (7043):['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
----------------------------------------
Column:InternetService
Unique values (3):['DSL' 'Fiber optic' 'No']
----------------------------------------
Column:Contract
Unique values (3):['Month-to-month' 'One year' 'Two year']
----------------------------------------
Column:PaymentMethod
Unique values (4):['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
----------------------------------------


I'll leave the rest of the columns as they are right now to preserve the information. 